In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

# Part 1
## Scraping table using BeautifulSoup#

In [2]:
#install Beautiful Soup and requests for Web Scaping\n",
!pip install BeautifulSoup4
!pip install requests

from bs4 import BeautifulSoup
import requests
        
#get from wiki page and create soup var
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')


#using soup object, iterate the .wikitable to get the data from the HTML page and store it into a list
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
for td in tr.find_all(['th','td']):
    section.append(td.text.rstrip())
    
    
#First row of data is the header
if (index == 0):
    columns = section
else:
    data.append(section)
print('done')

done


## import ssl and csv and urlopen

In [3]:
from urllib.request import urlopen
import ssl
import csv

print('csv imported.')

csv imported.


## wiki page and define soup variable

In [4]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
print('soup ready')

soup ready


## Convert to pandas dataframe

In [5]:
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')
ata = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  

print('Top five rows')
print(df.head(5))
print('Bottom five rows')
print(df.tail(5))

Top five rows
  PostalCode       Borough     Neighbourhood
0        M9Z  Not assigned      Not assigned
2        M1A  Not assigned      Not assigned
3        M2A  Not assigned      Not assigned
4        M3A    North York         Parkwoods
5        M4A    North York  Victoria Village
Bottom five rows
    PostalCode       Borough          Neighbourhood
284        M8Z     Etobicoke              Mimico NW
285        M8Z     Etobicoke     The Queensway West
286        M8Z     Etobicoke  Royal York South West
287        M8Z     Etobicoke         South of Bloor
288        M9Z  Not assigned           Not assigned


In [6]:
#determine size
df.shape

(288, 3)

## Process cells that have assigned boroughs, remove not assigned from Boroughs, redef Not Assigned for Neighbourhoods, and combine neighbourhoods with the same postal code

In [7]:
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'lxml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]

#drop not assigned Borough 
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df.loc[df['Neighbourhood']=="Not assigned",'Neighbourhood']=df.loc[df['Neighbourhood']=="Not assigned",'Borough']

# more than one neighbourhood in one postal code -> These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
df= df.groupby('PostalCode').agg(lambda x: ','.join(x))
df.head(20)

,Borough,Neighbourhood
PostalCode,,
M1B,"Scarborough,Scarborough","Rouge,Malvern"
M1C,"Scarborough,Scarborough,Scarborough","Highland Creek,Rouge Hill,Port Union"
M1E,"Scarborough,Scarborough,Scarborough","Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,"Scarborough,Scarborough,Scarborough","East Birchmount Park,Ionview,Kennedy Park"
M1L,"Scarborough,Scarborough,Scarborough","Clairlea,Golden Mile,Oakridge"
M1M,"Scarborough,Scarborough,Scarborough","Cliffcrest,Cliffside,Scarborough Village West"


## Reindex and see size

In [8]:
df1 = df.reset_index()
df1.shape

(103, 3)

# Part 2
## Adding address, coorindates, latitude, longitude, 

In [13]:
df1.to_csv('geo.csv')

In [64]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0


In [65]:
# The code was removed by Watson Studio for sharing.

In [66]:
body = client_ce6a4684c5c74247a3a6b23afaf8eb29.get_object(Bucket='week3ofcapstone-donotdelete-pr-mranwzywnl5qyv',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
df_data_1 = pd.read_csv(body)
df_data_1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [74]:
df_data_1.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_data_1.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
file = pd.merge(df1, df_data_1, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

file.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,"Scarborough,Scarborough","Rouge,Malvern",43.806686,-79.194353
1,M1C,"Scarborough,Scarborough,Scarborough","Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,"Scarborough,Scarborough,Scarborough","Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [75]:
#checking how Boroughs are listed
print(file['Borough'].unique())

['Scarborough' 'North York' 'East York' 'East Toronto' 'Central Toronto'
 'Downtown Toronto' 'York' 'West Toronto' "Queen's Park" 'Mississauga'
 'Etobicoke']


In [76]:
#cleaning Borough
file['Borough'] = file['Borough'].apply(lambda x: x.split(',')[0])

In [41]:
file.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [77]:
#making sure Boroughs is cleaned up
print(file['Borough'].unique())

['Scarborough' 'North York' 'East York' 'East Toronto' 'Central Toronto'
 'Downtown Toronto' 'York' 'West Toronto' "Queen's Park" 'Mississauga'
 'Etobicoke']


In [78]:
file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
Latitude         103 non-null float64
Longitude        103 non-null float64
dtypes: float64(2), object(3)
memory usage: 9.8+ KB


In [79]:
file.to_csv('geoll.csv')

# Part 3
## Explore and cluster the neighborhoods in Toronto

In [80]:
# load into json
file.to_json(path_or_buf='geo_tdot.json', orient='table')
with open('geo_tdot.json') as json_data:
    geo_tdot = json.load(json_data)
print('done')

done


In [81]:
geo_tdot

{'schema': {'fields': [{'name': 'index', 'type': 'integer'},
   {'name': 'PostalCode', 'type': 'string'},
   {'name': 'Borough', 'type': 'string'},
   {'name': 'Neighbourhood', 'type': 'string'},
   {'name': 'Latitude', 'type': 'number'},
   {'name': 'Longitude', 'type': 'number'}],
  'primaryKey': ['index'],
  'pandas_version': '0.20.0'},
 'data': [{'index': 0,
   'PostalCode': 'M1B',
   'Borough': 'Scarborough',
   'Neighbourhood': 'Rouge,Malvern',
   'Latitude': 43.8066863,
   'Longitude': -79.1943534},
  {'index': 1,
   'PostalCode': 'M1C',
   'Borough': 'Scarborough',
   'Neighbourhood': 'Highland Creek,Rouge Hill,Port Union',
   'Latitude': 43.7845351,
   'Longitude': -79.1604971},
  {'index': 2,
   'PostalCode': 'M1E',
   'Borough': 'Scarborough',
   'Neighbourhood': 'Guildwood,Morningside,West Hill',
   'Latitude': 43.7635726,
   'Longitude': -79.1887115},
  {'index': 3,
   'PostalCode': 'M1G',
   'Borough': 'Scarborough',
   'Neighbourhood': 'Woburn',
   'Latitude': 43.7709921

# explaining what I decided to do and reporting any observations

### checking size of data, choosing a randome field, make sure the data has all boroughs and neighbourhoods

In [82]:
#checking dimension
file.shape

(103, 5)

In [83]:
#Looking at a random item
some_data = geo_tdot['data']
some_data[99]

{'index': 99,
 'PostalCode': 'M9P',
 'Borough': 'Etobicoke',
 'Neighbourhood': 'Westmount',
 'Latitude': 43.696319,
 'Longitude': -79.5322424}

In [84]:
#how many unique boroughs
file['Borough'].nunique()

11

In [85]:
#making sure we have all the 11 boroughs and 103 neighbourhoods
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(file['Borough'].unique()),
        file.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


## generate maps to visualize neighborhoods and how they cluster together

In [86]:
#naming agent Toronto_explorer and finding the lat/long for Toronto
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [87]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(file['Latitude'], file['Longitude'], file['Borough'], file['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [88]:
# let's just look at Downtown Toronto

In [89]:
downtown_toronto = file[file['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [90]:
# getting the geolocation for Toronto
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [73]:
# create map of Tdot using latitude and longitude values
map_tdot = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_toronto['Latitude'], downtown_toronto['Longitude'], downtown_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tdot)  
    
map_tdot

<span style="color:purple"> **That's all for the 'week 3' assignment in the capstone course! _Thanks!_**</span>